In [10]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px


df = pd.read_stata('./fwd_holding_merged_10q1-23q2.dta')

df = df[[
    'seriescode',
    'currency',
    'yq',
    'curPur_usd',
    'curSold_usd',
    'netassets'
]]

df['net_forward_sale'] = (df['curSold_usd'] - df['curPur_usd']) / df['netassets']

# Group by currency and date, keep net only
df_plot = df.groupby(['currency', 'yq'])[['net_forward_sale']].sum().reset_index()


def currency_plot(df_plot):
    # Convert 'yq' to a quarterly period string for the bar chart legend.
    df_plot['select_date'] = pd.to_datetime(df_plot['yq'], format='%Y%m%d').dt.to_period('Q').astype(str)
    df_plot = df_plot.sort_values('select_date')

    # Create the bar chart
    bar_fig = px.bar(
        df_plot,
        x="net_forward_sale",
        y="currency",
        color="select_date",
        orientation="h",
        title="Net Forward Sale Position by Date"
    )
    
    # Set each trace to be initially hidden in the plot (i.e., non-selected).
    for trace in bar_fig.data:
        trace.visible = "legendonly"

    bar_fig.update_layout(
        xaxis_title="", 
        yaxis_title="Currency",
        template="plotly_white"
    )
    

    # bar_fig.write_json("bar_chart.json")
    bar_fig.write_html("bar_chart.html", include_plotlyjs="cdn")


    app = dash.Dash(__name__)

    # Define the layout
    app.layout = html.Div([
        html.H1("", style={'textAlign': 'center'}),

        # Section 1: Bar Chart 
        html.Div([
            html.H2("Net Position by Currency Across Dates"),
            dcc.Graph(id='bar-chart', figure=bar_fig)
        ], style={'marginBottom': 50, 'width': '90%', 'margin': 'auto'}),

        # Section 2: Area Chart
        html.Div([
            html.H2("Net Position by Currency Over Time"),
            dcc.Dropdown(
                id='currency-dropdown',
                options=[{'label': cur, 'value': cur} for cur in sorted(df_plot['currency'].unique())],
                value=sorted(df_plot['currency'].unique()),  
                multi=True,
                placeholder="Select currencies"
            ),
            dcc.Graph(id='area-plot')
        ], style={'width': '90%', 'margin': 'auto'})
    ])

    # Callback to update
    @app.callback(
        Output('area-plot', 'figure'),
        [Input('currency-dropdown', 'value')]
    )
    def update_area_chart(selected_currencies):
        filtered_df = df_plot[df_plot['currency'].isin(selected_currencies)] if selected_currencies else df_plot.iloc[0:0]
        area_fig = px.area(filtered_df, x="yq", y="net_forward_sale", color="currency")
        area_fig.update_layout(
            xaxis_title="",
            yaxis_title="Net Forward Sale Position by Currency",
            template="plotly_white"
        )
        
        '''
        # Optional: Also set each trace to be initially hidden
        for trace in area_fig.data:
            trace.visible = "legendonly"
        '''
        # area_fig.write_json("area_chart.json")
        area_fig.write_html("area_chart.html", include_plotlyjs="cdn")
        return area_fig

    if __name__ == '__main__':
        app.run_server(debug=True)



currency_plot(df_plot)

